### Caterpillar algorithm:

In [1]:
import random
from random import randint
import numpy as np

class Caterpillar:
    numInput = 0
    weights = []
    threshold = 0.0
    alpha = 0.0
    alpha2 = 0.0
    
    def __init__(self, numInput, rndSeed):
        self.numInput = numInput
        self.weights = [0] *numInput
        for i in range(len(self.weights)):
            self.weights[i] = numInput / 2.0
        self.threshold = 1.0 * numInput
        self.alpha = 2.0
        self.alpha2 = 4.0
        random.seed( rndSeed )
        
    
    def ComputeY(self, xValues):
        sum = 0.0
        for i in range(self.numInput):
            sum += self.weights[i] * xValues[i]
        if sum > self.threshold:
            return 1
        else:
            return 0
     
    
    def ShuffleObservations(self, trainData):
        for i in range(len(trainData)):
            r = randint(i, len(trainData) - 1)
            tmp = []
            tmp = trainData[r]
            trainData[r] = trainData[i]
            trainData[i] = tmp
         
    def Accuracy(self, trainData):
        numCorrect = 0
        numWrong = 0
        xValues = [0] *self.numInput
        
        for i in range(len(trainData)):
            xValues = np.copy(trainData[i])
            target = trainData[i][self.numInput]
            computed = self.ComputeY(xValues)

            if computed == target:
                numCorrect += 1
            else:
                numWrong += 1
                
        return (numCorrect * 1.0) / (numCorrect + numWrong)
    
    
    def TrainWeights(self, trainData):
        xValues = [] * self.numInput
        self.ShuffleObservations(trainData)
        
        for i in range(len(trainData)):
            #  get the inputs
            xValues = np.copy(trainData[i])
            
            #  last value is target
            target = trainData[i][self.numInput] 
            
            # predict
            computed = self.ComputeY(xValues)

            # train
            if (computed == 1 and target == 0):
                # need to decrease weight:
                for j in range(self.numInput):
                    if (xValues[j] == 0): continue
                    self.weights[j] = self.weights[j] / self.alpha2 
            elif (computed == 0 and target == 1):
                # need to increase weight:
                for j in range(self.numInput):
                    if (xValues[j] == 0): continue
                    self.weights[j] = self.weights[j] * self.alpha

        result = [0.0] *self.numInput
        result = self.weights
        
        return result

Increased the training weights iterations by 150 and made sure that the variables are updated properly.

In [2]:
def ShowVector(vector, decimals, valsPerRow, newLine):
    frmt = '%.' + str(decimals) + 'f'
    for i in range(len(vector)):
        if (i % valsPerRow == 0): print("", end='')
        print(frmt % vector[i] + " ", end='')
    if (newLine): print("")


def ShowMatrix(matrix, decimals, numRows, indices):
    frmt = '%.' + str(decimals) + 'f'
    for i in range(numRows):
        if (indices):
            print("[" + '%02d' % i + "]   ", end='')
        for j in range(len(matrix[i])):
            print(frmt % matrix[i][j] + " ", end='')
        print("")
    lastIndex = len(matrix) - 1
    if (indices):
        print("[" + '%02d' % lastIndex + "]   ", end='')
    for j in range(len(matrix[lastIndex])):
        print(frmt % matrix[lastIndex][j] + " ", end='')
    print("")

In [3]:
import numpy as np

def MakeTrainTest(data, pct, seed):
    if 1 < pct:
        print("The percentage should be specified as a number less than 1!")
        return None
    
    totRows = data.shape[0] #compute number of rows in each result
    numTrainRows = int(totRows * pct)
    numTestRows = totRows - numTrainRows
    #trainData = new int[numTrainRows][]
    trainData = np.empty(data.shape)
    #testData = new int[numTestRows][]
    testData = np.empty(data.shape)
    copy = np.empty(data.shape)

    # int[][] copy = new int[data.Length][] #  make a copy of data
    for i in range(copy.shape[0]):
        # by reference to save space
        copy[i] = data[i]
    for i in range(copy.shape[0]):
        # scramble row order of copy
        r = randint(i, copy.shape[0] - 1)
        tmp = copy[r]
        copy[r] = copy[i]
        copy[i] = tmp
    for i in range(numTrainRows):
        # create training
        trainData[i] = copy[i]
    for i in range(numTestRows):
        # create test
        testData[i] = copy[i + numTrainRows]
        
    return trainData, testData

In [4]:
import numpy as np
data = np.empty((100,17))

data[0] =  [ 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1 ] #  last col is the label
data[1] =  [ 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1 ]
data[2] =  [ 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0 ]
data[3] =  [ 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0 ]
data[4] =  [ 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0 ]
data[5] =  [ 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0 ]
data[6] =  [ 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0 ]
data[7] =  [ 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1 ]
data[8] =  [ 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1 ]
data[9] =  [ 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0 ]
data[10] = [ 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1 ]
data[11] = [ 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1 ]
data[12] = [ 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0 ]
data[13] = [ 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0 ]
data[14] = [ 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1 ]
data[15] = [ 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1 ]
data[16] = [ 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0 ]
data[17] = [ 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0 ]
data[18] = [ 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1 ]
data[19] = [ 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0 ]
data[20] = [ 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0 ]
data[21] = [ 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0 ]
data[22] = [ 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0 ]
data[23] = [ 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0 ]
data[24] = [ 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0 ]
data[25] = [ 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0 ]
data[26] = [ 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0 ]
data[27] = [ 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0 ]
data[28] = [ 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1 ]
data[29] = [ 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0 ]
data[30] = [ 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1 ]
data[31] = [ 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0 ]
data[32] = [ 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0 ]
data[33] = [ 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1 ]
data[34] = [ 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0 ]
data[35] = [ 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1 ]
data[36] = [ 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1 ]
data[37] = [ 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1 ]
data[38] = [ 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1 ]
data[39] = [ 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0 ]
data[40] = [ 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0 ]
data[41] = [ 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0 ]
data[42] = [ 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0 ]
data[43] = [ 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0 ]
data[44] = [ 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0 ]
data[45] = [ 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0 ]
data[46] = [ 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0 ]
data[47] = [ 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0 ]
data[48] = [ 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0 ]
data[49] = [ 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1 ]
data[50] = [ 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0 ]
data[51] = [ 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1 ]
data[52] = [ 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0 ]
data[53] = [ 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1 ]
data[54] = [ 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0 ]
data[55] = [ 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1 ]
data[56] = [ 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1 ]
data[57] = [ 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1 ]
data[58] = [ 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1 ]
data[59] = [ 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1 ]
data[60] = [ 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0 ]
data[61] = [ 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1 ]
data[62] = [ 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0 ]
data[63] = [ 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0 ]
data[64] = [ 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0 ]
data[65] = [ 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1 ]
data[66] = [ 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1 ]
data[67] = [ 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1 ]
data[68] = [ 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0 ]
data[69] = [ 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0 ]
data[70] = [ 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0 ]
data[71] = [ 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1 ]
data[72] = [ 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0 ]
data[73] = [ 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1 ]
data[74] = [ 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0 ]
data[75] = [ 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0 ]
data[76] = [ 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0 ]
data[77] = [ 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0 ]
data[78] = [ 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0 ]
data[79] = [ 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1 ]
data[80] = [ 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0 ]
data[81] = [ 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0 ]
data[82] = [ 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1 ]
data[83] = [ 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1 ]
data[84] = [ 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1 ]
data[85] = [ 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0 ]
data[86] = [ 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1 ]
data[87] = [ 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1 ]
data[88] = [ 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0 ]
data[89] = [ 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1 ]
data[90] = [ 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0 ]
data[91] = [ 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0 ]
data[92] = [ 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0 ]
data[93] = [ 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0 ]
data[94] = [ 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0 ]
data[95] = [ 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0 ]
data[96] = [ 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0 ]
data[97] = [ 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0 ]
data[98] = [ 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0 ]
data[99] = [ 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1 ]

In [5]:
import pandas as pd
caterpillar_notebook = pd.DataFrame(data)
caterpillar_notebook.head(20)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0
1,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0
2,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0
3,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0
4,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0
5,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0
6,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0
7,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0
8,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0
9,1.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
print("First few lines of all data are:")
ShowMatrix(data, 0, 6, True)

First few lines of all data are:
[00]   0 1 0 1 1 1 0 0 0 1 0 1 1 1 0 1 1 
[01]   0 1 0 1 1 1 0 0 0 0 0 1 1 1 0 0 1 
[02]   0 1 1 0 1 1 0 0 0 0 1 0 1 1 0 0 0 
[03]   0 1 1 0 0 1 0 0 0 0 1 0 1 0 0 1 0 
[04]   1 1 1 0 1 1 0 0 0 0 1 0 1 1 1 1 0 
[05]   0 1 1 0 1 1 0 0 0 0 0 0 1 1 1 1 0 
[99]   0 0 0 1 1 1 0 0 0 1 0 1 1 1 0 0 1 


In [7]:
print("Splitting data into 80% train and 20% test matrices")
trainData, testData = MakeTrainTest(data, 0.8, 17)

Splitting data into 80% train and 20% test matrices


In [8]:
print("First few rows of training data are:")
ShowMatrix(trainData, 0, 6, True)

First few rows of training data are:
[00]   0 1 0 1 1 1 0 0 0 1 0 1 1 1 0 1 1 
[01]   0 1 0 1 1 1 0 0 0 0 0 1 1 1 0 0 1 
[02]   0 1 1 0 1 1 0 0 0 0 1 0 1 1 0 0 0 
[03]   0 1 1 0 0 1 0 0 0 0 1 0 1 0 0 1 0 
[04]   1 1 1 0 1 1 0 0 0 0 1 0 1 1 1 1 0 
[05]   0 1 1 0 1 1 0 0 0 0 0 0 1 1 1 1 0 
[99]   0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 


In [9]:
print("First few rows of testing data are:")
ShowMatrix(testData, 0, 6, True)

First few rows of testing data are:
[00]   1 1 1 0 0 0 1 1 0 0 1 0 0 0 1 1 0 
[01]   0 1 0 1 1 1 0 0 0 0 0 1 1 1 0 1 1 
[02]   1 0 1 0 0 0 1 1 1 1 0 0 0 0 1 1 0 
[03]   1 0 1 0 0 0 1 1 1 1 0 0 0 0 1 1 0 
[04]   1 1 1 0 0 0 1 1 1 0 1 0 0 0 1 1 0 
[05]   1 0 1 0 0 1 1 1 1 1 1 0 0 1 1 1 0 
[99]   0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 


In [10]:
%%time

print("Begin training using Winnow algorithm")
numInput = 16
w = Caterpillar(numInput, 19) #rndSeed = 0
for i in range(150):
    weights = w.TrainWeights(trainData)
print("Training complete")

Begin training using Winnow algorithm
Training complete
CPU times: user 152 ms, sys: 8.45 ms, total: 160 ms
Wall time: 155 ms


In [11]:
print("Final model weights are:")
ShowVector(weights, 4, 8, True)

Final model weights are:
0.1250 0.0625 0.0005 16.0000 1.0000 0.0625 2.0000 0.0312 0.1250 0.5000 0.0156 1.0000 0.1250 0.5000 2.0000 0.0625 


In [12]:
trainAcc = w.Accuracy(trainData)
testAcc = w.Accuracy(testData)

print("Prediction accuracy on training data = " + str(trainAcc))
print("Prediction accuracy on test data = " + str(testAcc))

Prediction accuracy on training data = 1.0
Prediction accuracy on test data = 1.0


Results: Obtained prediction accuracy as 100% on test data.